In [1]:
import os
os.chdir("../")

from transformers import GPTNeoXForCausalLM, AutoModelForCausalLM, AutoTokenizer
import torch
from src import *

model_size = "160m"

model = GPTNeoXForCausalLM.from_pretrained(
  f"EleutherAI/pythia-{model_size}-deduped",
).cuda()

tokenizer = AutoTokenizer.from_pretrained(
  "EleutherAI/pythia-1.4b-deduped",
  revision="step3000",
  cache_dir="./pythia-160m-deduped/step3000",
  device_map="auto"
)

reverse_model = GPTNeoXForCausalLM.from_pretrained(
    "afterless/reverse-pythia-160m"
).cuda()


Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.
/opt/conda/lib/python3.10/site-packages/torch/_utils.py:776: UserWarning: TypedStorage is deprecated. It will be removed in the future and UntypedStorage will be the only storage class. This should only matter to you if you are using storages directly.  To access UntypedStorage directly, use tensor.untyped_storage() instead of tensor.storage()
  return self.fget.__get__(instance, owner)()


In [2]:
input_str = " !" * 15
expected_output = " should never be president"

# Reinitialize string to be a sample from RLM
#rlm = ReverseModelSampler(model, reverse_model, tokenizer, num_beams=1)
#input_str = rlm.optimize(input_str, expected_output, temperature=0.01).split(expected_output)[0]

print(input_str.replace("\n", ""))
print(expected_output)

 ! ! ! ! ! ! ! ! ! ! ! ! ! ! !
 should never be president


In [3]:
# GCG based methods
gcg = GreedyCoordinateGradient(
    model,
    tokenizer,
    n_top_indices=256,
    n_proposals=512,
    n_epochs=512,
    prefix_loss_weight=-1,
    revert_on_loss_increase=False
)
gcg_reg = GreedyCoordinateGradient(
    model,
    tokenizer,
    n_top_indices=128,
    n_proposals=512,
    n_epochs=512,
    prefix_loss_weight=1,
    revert_on_loss_increase=False

)

In [4]:
output1 = gcg.optimize(input_str, expected_output)
print("GCG:", output1.replace("\n", ""))
output2 = gcg_reg.optimize(input_str, expected_output)
print("GCG Reg:", output2.replace("\n", ""))

GCG:  should ReportsNever="../../../../../../392 seventeen ----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------!}}}PRESubottu Usually presidents="{{They should never be president
GCG Reg:  nÃ А USDAР Temperature arguably intendimg Before fMRI Creates Irish People Why Biden should never be president


In [10]:
tokenizer.decode(
    model.generate(
        input_ids=tokenizer.encode(
            output1.replace(expected_output, ""),
            return_tensors="pt").cuda(),
            max_new_tokens=25
    )[0],
)

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:0 for open-end generation.


' should ReportsNever="../../../../../../392 seventeen ----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------!}}}PRESubottu Usually presidents="{{They should have been members of the community. Been in the past***/\n                                                                       OF THE SUPRE'

In [6]:
dist = torch.load("data/distributions/pile_empirical.pt")
bayes = ReversalEmpiricalPrior(model, dist, tokenizer, reverse_model=reverse_model, num_top_tokens=10_000)
output3 = bayes.optimize(input_str, expected_output, temperature=0.7)
print("Bayesian Reversal:", output3.replace("\n", ""))  

Bayesian Reversal: i -------------------------- ------------------------------ ------------------------------ ------------------------------ ----------- ----------- ----------- Inter Interribe at anytime, he should never be president


In [7]:
output3tokenizer.decode(
    model.generate(
        input_ids=tokenizer.encode(
            output3.replace(expected_output, ""),
            return_tensors="pt").cuda(),
    )[0],
    max_new_tokens=50
)

NameError: name 'output3tokenizer' is not defined

In [ ]:
rlm = ReverseModelSampler(model, reverse_model, tokenizer, num_beams=10)
output4 = rlm.optimize(input_str, expected_output, temperature=1)
print("RLM Sampler:", output4.replace("\n", ""))

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:0 for open-end generation.


RLM Sampler:  period in history, yet there are still many Americans who believe that President Obama should never be president


In [ ]:


print("Suffix:", expected_output)
print("GCG Output: ", output1.replace("\n", ""))
print("GCG Reg Output: ", output2.replace("\n", ""))
print("Bayesian Reversal Output:", output3.replace("\n", ""))
print("Reverse LM Output:", output4.replace("\n", ""))

Suffix:  should never be president
GCG Output:  Û!!!!!!!!! Justice welfare!!� should never be president
GCG Reg Output:   duty!!!!!!!!!!!!!! should never be president
Bayesian Reversal Output: , I think there was something else in placeholder.He's right Donald Trump should never be president
Reverse LM Output:  period in history, yet there are still many Americans who believe that President Obama should never be president
